In [ ]:
import altair as alt
from src.visualization.common import setup_altair
setup_altair()

from src.inference.mcmc.sample_containers import DoublingSampleContainer

class DummyState:

    def __init__(self):
        self.i = -1

    def do_something(self):
        self.i += 1

    def get_state(self):
        return f"State: {self.i}"

In [ ]:
cont = DoublingSampleContainer(10)
model = DummyState()

obs = []
for i in range(500):

    model.do_something()
    cont.register_sample(lambda : model.get_state())
    obs.append({"step" : i, "kept": list(cont.samples)})


In [ ]:
import pandas as pd
import seaborn as sns

base = (
    pd.DataFrame(obs)
    .explode("kept")
    .groupby("kept")
    .agg(["min", "max"])
    .step
    .reset_index()
    .melt(id_vars="kept")
    .pipe(alt.Chart)
)

bars = (
    base
    .mark_line()
    .encode(
        x="kept",
        y=alt.Y("value", scale=alt.Scale(reverse=True)),
        detail="kept"
    )
)


points = (
    base
    .mark_point()
    .encode(
        x="kept",
        y=alt.Y("value", scale=alt.Scale(reverse=True))
    )

)
points + bars

In [ ]:
points = (
    base
    .transform_fold(["min", "max"])
    .mark_point()
    .encode(
        x="kept",
        y=alt.Y("value", scale=alt.Scale(reverse=True)),
        color="key"
    )
)
points